# advent of code 2024 - [day 2](https://adventofcode.com/2024/day/2)

In [4]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [5]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

2.12.0


In [6]:
def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
            any_update = any_update or result.consume().counters._contains_updates
        if not any_update:
            break

In [7]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for _, line in enumerate(file):
        els = line.strip().split(" ")
        yield list(int(el) for el in els)

## Python-based solution

### Part 1

In [8]:
def is_safely_increasing(l):
    if len(l) < 2:
        return True
    return l[-2]+1 <= l[-1] <= l[-2]+3 and is_safely_increasing(l[:-1])


def is_safely_decreasing(l):
    return is_safely_increasing(list(reversed(l)))

def is_safely_monotonous(l):
    return is_safely_increasing(l) or is_safely_decreasing(l)

In [9]:
monotonous_lists = (l for l in gen_lists() if is_safely_monotonous(l) )
part1 = len(list(monotonous_lists))
part1


526

### Part 2

In [10]:
def is_safely_increasing(l):
    if len(l) < 2:
        return True
    return l[-2]+1 <= l[-1] <= l[-2]+3 and is_safely_increasing(l[:-1])

def is_safely_increasing_with_tolerance(l):
    return any(is_safely_increasing(l[:ix]+l[ix+1:]) for ix, _ in enumerate(l))

def is_safely_decreasing_with_tolerance(l):
    return is_safely_increasing_with_tolerance(list(reversed(l)))

def is_safely_monotonous_with_tolerance(l):
    return is_safely_increasing_with_tolerance(l) or is_safely_decreasing_with_tolerance(l)

In [11]:
monotonous_lists = (l for l in gen_lists() if is_safely_monotonous_with_tolerance(l) )
part2 = len(list(monotonous_lists))
part2

566

## Neo4j-based solution

### Parsing

In [23]:
reports = list(gen_lists())

### Ingestion

In [24]:
query_ingest = """
UNWIND range(0, size($reports)-1) AS rep_ix
UNWIND range(0, size($reports[rep_ix])-1) AS rec_ix
WITH rep_ix AS report_id, rec_ix AS record_number, $reports[rep_ix][rec_ix] AS value
CREATE (:Record {report_id: report_id, record_number: record_number, value: value})
"""

gds.run_cypher(query_ingest, {"reports":reports})

""


### Chaining

In [25]:
query_chaining = """
MATCH (x:Record)
WITH x ORDER BY x.record_number
WITH x.report_id AS report_id, collect(x) AS xs
CALL apoc.nodes.link(xs, 'NEXT');
"""

gds.run_cypher(query_chaining, {})

""


### Query

In [26]:
gds.run_cypher('CREATE INDEX record_val IF NOT EXISTS FOR (r:Record) ON (r.value)')
gds.run_cypher('CREATE INDEX record_record_number IF NOT EXISTS FOR (r:Record) ON (r.record_number)');

In [27]:
query_part1 = """
MATCH (first:Record {record_number: 0})-->(second)
WHERE first.value <> second.value
WITH first, second, (second.value-first.value)/abs(second.value-first.value) AS sign
MATCH path = (first)(
  (x)-->(y)
  WHERE 1 <= sign * (y.value - x.value) <= 3
)*(last WHERE NOT EXISTS {(last)-->()})
RETURN count(path) AS part1
"""

gds.run_cypher(query_part1, {})

,part1
0,526


In [29]:
query_part2 = """
MATCH (first:Record {record_number: 0})-->(second)
WHERE first.value <> second.value
WITH first, second, (second.value-first.value)/abs(second.value-first.value) AS sign
MATCH path = (first)(
  (x)-->(y)
  WHERE 1 <= sign * (y.value - x.value) <= 3
)*(last WHERE NOT EXISTS {(last)-->()})
WITH first, apoc.coll.sum([x IN apoc.coll.zip(x,y)|
  CASE 1 <= sign *(x[1].value - x[0].value) <= 3 WHEN False THEN 1 ELSE 0 END]) AS anomaly
WHERE anomaly <= 1
RETURN count(first) AS part2
"""

gds.run_cypher(query_part2, {})

,part2
0,526
